In [1]:
import numpy as np
import pandas as pd

from data import *
from parameters import *
from utils import *

In [2]:
NUM_SAMPLES = 10000

# Hardware acquisition cost

ai_accelerator_cost = hardware_quantity * hardware_price

server_cost = ai_accelerator_cost * server_cost_overhead

cluster_cost = server_cost * cluster_interconnect_overhead

90% credible intervals:
- hardware_quantity: 0.8x to 1.25x of the central value. Generally confident, as it was usually reported directly by the developers. Could be slightly inaccurate if it doesn't account for hardware failures during training, or it describes the number of chips available in a cluster rather than the number of chips actually employed for training.
- hardware_price: 0.5x to 2x of the central value. We know of two reports of NVIDIA's profit margin for Hopper GPUs: 1000% and 340%. So the minimum sale price could be 10 to 30% of the value we used. But this excludes R&D cost, and there should always be some profit, so we set the minimum higher than this. Big customers could be getting much bigger discounts compared to the prices we use. On the other hand, demand could have led to price spikes that weren't captured in our sparse data.
- server_cost_overhead: 1.3 to 2.1. The three actual values we calculated ranged from 1.54 (P100) to 1.69 (V100). A cost breakdown of a DGX H100 by Semianalysis (2023) https://www.semianalysis.com/p/ai-server-cost-analysis-memory-is implies a ratio of approximately 1.4 (total cost divided by "8 GPU + 4 NVSwitch Baseboard" cost).
- cluster_interconnect_overhead: 1.07 to 1.32. Prices of interconnect components are plausibly off by a factor of 2x due to variation in the brand, the merchant, and supply/demand.

In [3]:
# tpus
hardware_quantity = lognorm_from_90_ci(0.8, 1.25, NUM_SAMPLES)
hardware_price = lognorm_from_90_ci(0.5, 2, NUM_SAMPLES)
server_cost_overhead = lognorm_from_90_ci(1.3, 2.1, NUM_SAMPLES)
cluster_interconnect_overhead = lognorm_from_90_ci(1.07, 1.32, NUM_SAMPLES)
hardware_acquisition_cost = hardware_quantity * hardware_price * server_cost_overhead * cluster_interconnect_overhead
print_median_and_ci(hardware_acquisition_cost / np.median(hardware_acquisition_cost))
print_median_and_ci(hardware_acquisition_cost / np.median(hardware_acquisition_cost), ci=[25, 75])


Median: 1 [90% CI: 0.46, 2.1]
Median: 1 [50% CI: 0.73, 1.4]


In [4]:
# TPUs
hardware_quantity = lognorm_from_90_ci(0.8, 1.25, NUM_SAMPLES)

# Best estimate: ~$5,500
# Manufacturing cost based on H100: ~2,500
# Equivalent tpu price: ~11,000
# Go even wider than that, since the methodology for the bounds is limited too
# We saw manufacturing cost estimates as low as ~$1,000
# R&D costs would still add to that. Overall, a factor of 4 uncertainty seems reasonable.
hardware_price = lognorm_from_90_ci(0.25, 4, NUM_SAMPLES)

server_cost_overhead = lognorm_from_90_ci(1.3, 2.1, NUM_SAMPLES)
cluster_interconnect_overhead = lognorm_from_90_ci(1.07, 1.32, NUM_SAMPLES)
hardware_acquisition_cost = hardware_quantity * hardware_price * server_cost_overhead * cluster_interconnect_overhead
print_median_and_ci(hardware_acquisition_cost / np.median(hardware_acquisition_cost))
print_median_and_ci(hardware_acquisition_cost / np.median(hardware_acquisition_cost), ci=[25, 75])

Median: 1 [90% CI: 0.23, 4.2]
Median: 1 [50% CI: 0.54, 1.8]


# Amortized hardware CapEx

depreciation = 10 ** (ML_tpu_PRICE_PERFORMANCE_OOMS_PER_YEAR * years_since)

training_chip_hours = hardware_quantity * training_time

OR

training_chip_hours = training_compute / (peak_flop_per_second * hardware_utilization)

In the amortized hardware CapEx + energy approach, 32 values use the first method, 15 values use the second method.

amortized_hardware_capex = server_cost_overhead * cluster_interconnect_overhead * (hardware_price / depreciation) * ML_GPU_PRICE_PERFORMANCE_OOMS_PER_YEAR * np.log(10) / HOURS_PER_YEAR * training_chip_hours

90% credible intervals:

- ML_GPU_PRICE_PERFORMANCE_OOMS_PER_YEAR: 0.10 to 0.18, based on Hobbhahn et al. (2023)
- years_since: -120 days or +120 days from the central value.
  - years_since measures the difference between the time the hardware could first be acquired at large scale, and the model training start date. The default is at least 90 days between hardware release date and hardware acquisition date, and 60 days + training time between training start date and publication date. However, shipping could be almost immediate relative to the release date, especially if customers can pre-order. Preparing results after training may only take about 30 days, especially if the evaluations are ready to run and the model is announced before a detailed report is released. In total that's (90 - 0) + (60 - 30) = 120 fewer days than the default. We keep the bounds symmetric to fit a normal distribution, since years_since is used as an exponent in the formula.
- training_time: 0.5x to 2x of the central value. It was usually reported directly by the developers, but sometimes it's reported imprecisely - e.g. "2 days" is plausibly anywhere between 24 and 72 hours; "weeks" could mean 2 weeks or 6 weeks.
- training_compute: 0.33x to 3x of the central value - a rule of thumb based on the "Confident" Confidence level in the database.
- peak_flop_per_second: 0.5 to 2x of the central value. In case we got the number format wrong.
- hardware_utilization: normal, 0.1 to 0.65. The range of values in our dataset is 0.19 to 0.56; we go a bit wider to be conservative.

In [5]:
num_ml_models = 45
num_with_imputed_chip_hours = int(15/num_ml_models * NUM_SAMPLES)
num_with_direct_chip_hours = NUM_SAMPLES - num_with_imputed_chip_hours

In [6]:
def amortized_hardware_capex_sample(hardware_type='GPU'):    
    price_performance = norm_from_ci(0.10, 0.18, 90, NUM_SAMPLES)
    years_since = norm_from_ci(-120, 120, 90, NUM_SAMPLES) / DAYS_PER_YEAR
    depreciation = 10 ** (price_performance * years_since)

    if hardware_type == 'TPU':
        # Best estimate: ~$5,500
        # Manufacturing cost based on H100: ~2,500
        # Equivalent GPU price: ~11,000
        # Go even wider than that, since the methodology for the bounds is limited too
        # We saw manufacturing cost estimates as low as ~$1,000
        # R&D costs would still add to that. Overall, a factor of 4 uncertainty seems reasonable.
        hardware_price = lognorm_from_90_ci(0.25, 4, NUM_SAMPLES)
    else:
        hardware_price = lognorm_from_90_ci(0.5, 2, NUM_SAMPLES)

    server_cost_overhead = lognorm_from_90_ci(1.3, 2.1, NUM_SAMPLES)
    cluster_interconnect_overhead = lognorm_from_90_ci(1.07, 1.32, NUM_SAMPLES)

    training_compute = lognorm_from_90_ci(1/3, 3, num_with_imputed_chip_hours)
    peak_flop_per_second = lognorm_from_90_ci(0.5, 2, num_with_imputed_chip_hours)
    hardware_utilization_raw = norm_from_ci(0.1, 0.65, 90, num_with_imputed_chip_hours, clip=[0.01, 0.99])
    # Normalize by average value so the scale is consistent with direct_chip_hours
    hardware_utilization = hardware_utilization_raw / 0.37
    realised_flop_per_second = peak_flop_per_second * hardware_utilization
    imputed_chip_hours = training_compute / realised_flop_per_second

    hardware_quantity = lognorm_from_90_ci(0.8, 1.25, num_with_direct_chip_hours)
    training_time = lognorm_from_90_ci(0.5, 2, num_with_direct_chip_hours)
    direct_chip_hours = training_time * hardware_quantity

    training_chip_hours = np.concatenate([imputed_chip_hours, direct_chip_hours])
    amortized_hardware_capex = server_cost_overhead * cluster_interconnect_overhead * (hardware_price / depreciation) * price_performance * training_chip_hours

    return amortized_hardware_capex

In [7]:
amortized_hardware_capex_gpu = amortized_hardware_capex_sample(hardware_type='GPU')
print_median_and_ci(amortized_hardware_capex_gpu / np.median(amortized_hardware_capex_gpu))
print_median_and_ci(amortized_hardware_capex_gpu / np.median(amortized_hardware_capex_gpu), ci=[25, 75])


Median: 1 [90% CI: 0.27, 4]
Median: 1 [50% CI: 0.6, 1.7]


In [8]:
amortized_hardware_capex_tpu = amortized_hardware_capex_sample(hardware_type='TPU')
print_median_and_ci(amortized_hardware_capex_tpu / np.median(amortized_hardware_capex_tpu))
print_median_and_ci(amortized_hardware_capex_tpu / np.median(amortized_hardware_capex_tpu), ci=[25, 75])

Median: 1 [90% CI: 0.16, 6.6]
Median: 1 [50% CI: 0.49, 2]


# Energy

Energy cost rate ($/kWh) ∗ Hardware peak TDP (kW) ∗ Average power to TDP ratio (%) ∗ Data center PUE ∗ Number of chip-hours (h)

- Energy cost rate: 0.6x to 1.7x of central value (based on variation among likely data center states; see code below)
- Hardware peak TDP (kW): assumed accurate based on hardware specifications.
- Average power to TDP ratio: 
  - TPUs: 0.3 to 0.62
  - GPUs: 0.56 to 1
- Data center PUE
  - 1.05 to 1.16 for hyperscalers (50 of 64 selected models)
  - 1.12 to 1.4 for non-hyperscalers

Uncertainty in energy prices

In [9]:
from energy import *
likely_datacenter_states = ['California', 'Nevada', 'Oregon', 'Washington']
energy_prices = [US_STATE_ENERGY_PRICES_PER_KWH[state] for state in likely_datacenter_states]
mean = np.mean(energy_prices)
np.max(energy_prices) / mean, np.min(energy_prices) / mean

(1.727914785226352, 0.6247539654972791)

In [10]:
num_hyperscalers = int(50/64 * NUM_SAMPLES)
num_non_hyperscalers = NUM_SAMPLES - num_hyperscalers

In [11]:
def energy_sample(hardware_type='GPU'):
    training_compute = lognorm_from_90_ci(1/3, 3, num_with_imputed_chip_hours)
    peak_flop_per_second = lognorm_from_90_ci(0.5, 2, num_with_imputed_chip_hours)
    hardware_utilization_raw = norm_from_ci(0.1, 0.65, 90, num_with_imputed_chip_hours, clip=[0.01, 0.99])
    # Normalize by average value so the scale is consistent with direct_chip_hours
    hardware_utilization = hardware_utilization_raw / 0.37
    realised_flop_per_second = peak_flop_per_second * hardware_utilization
    imputed_chip_hours = training_compute / realised_flop_per_second

    hardware_quantity = lognorm_from_90_ci(0.8, 1.25, num_with_direct_chip_hours)
    training_time = lognorm_from_90_ci(0.5, 2, num_with_direct_chip_hours)
    direct_chip_hours = training_time * hardware_quantity

    training_chip_hours = np.concatenate([imputed_chip_hours, direct_chip_hours])

    avg_power_ratio_tpu_raw = lognorm_from_90_ci(0.3, 0.62, NUM_SAMPLES)
    # Normalize by average value so the scale is consistent
    avg_power_ratio_tpu = avg_power_ratio_tpu_raw / 0.43
    avg_power_ratio_gpu_raw = lognorm_from_90_ci(0.56, 1.00, NUM_SAMPLES)
    avg_power_ratio_gpu = avg_power_ratio_gpu_raw / 0.75
    if hardware_type == 'TPU':
        avg_power_ratio = avg_power_ratio_tpu
    else:
        avg_power_ratio = avg_power_ratio_gpu

    pue_hyperscalers_raw = lognorm_from_90_ci(1.05, 1.16, num_hyperscalers)
    pue_hyperscalers = pue_hyperscalers_raw / 1.1
    pue_non_hyperscalers_raw = lognorm_from_90_ci(1.12, 1.4, num_non_hyperscalers)
    pue_non_hyperscalers = pue_non_hyperscalers_raw / 1.25
    pue = np.concatenate([pue_hyperscalers, pue_non_hyperscalers])

    tdp = 1

    energy_price = lognorm_from_90_ci(0.6, 1.7, NUM_SAMPLES)

    energy = energy_price * tdp * avg_power_ratio * pue * training_chip_hours

    return energy

In [12]:
energy_tpu = energy_sample(hardware_type='TPU')
print_median_and_ci(energy_tpu / np.median(energy_tpu))
print_median_and_ci(energy_tpu / np.median(energy_tpu), ci=[25, 75])

Median: 1 [90% CI: 0.31, 3.6]
Median: 1 [50% CI: 0.63, 1.6]


In [13]:
energy_gpu = energy_sample(hardware_type='GPU')
print_median_and_ci(energy_gpu / np.median(energy_gpu))
print_median_and_ci(energy_gpu / np.median(energy_gpu), ci=[25, 75])

Median: 1 [90% CI: 0.31, 3.6]
Median: 1 [50% CI: 0.64, 1.6]


# Amortized hardware CapEx + energy

- Energy is between about 3% and 30% of amortized hardware CapEx + energy cost

In [14]:
# Fraction of total
energy_fraction = lognorm_from_90_ci(0.03, 0.3, NUM_SAMPLES)
# Fraction of capex
energy_fraction = energy_fraction / (1 - energy_fraction)
print_median_and_ci(energy_fraction)

Median: 0.11 [90% CI: 0.031, 0.42]


In [15]:
energy_fraction = lognorm_from_90_ci(0.03, 0.42, NUM_SAMPLES)

In [16]:

amortized_hardware_capex_energy_gpu = amortized_hardware_capex_gpu + energy_gpu * energy_fraction
amortized_hardware_capex_energy_gpu_proportion = amortized_hardware_capex_energy_gpu / np.median(amortized_hardware_capex_energy_gpu)
print_median_and_ci(amortized_hardware_capex_energy_gpu_proportion)
print_median_and_ci(amortized_hardware_capex_energy_gpu_proportion, ci=[25, 75])

Median: 1 [90% CI: 0.34, 3.7]
Median: 1 [50% CI: 0.65, 1.6]


In [17]:

amortized_hardware_capex_energy_tpu = amortized_hardware_capex_tpu + energy_tpu * energy_fraction
amortized_hardware_capex_energy_tpu_proportion = amortized_hardware_capex_energy_tpu / np.median(amortized_hardware_capex_energy_tpu)
print_median_and_ci(amortized_hardware_capex_energy_tpu_proportion)
print_median_and_ci(amortized_hardware_capex_energy_tpu_proportion, ci=[25, 75])

Median: 1 [90% CI: 0.27, 4.9]
Median: 1 [50% CI: 0.58, 1.8]
